In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10,6)
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics


%matplotlib inline

In [19]:
#load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0   1          60       RL  ...        WD         Normal    208500
1   2          20       RL  ...        WD         Normal    181500
2   3          60       RL  ...        WD         Normal    223500
3   4          70       RL  ...        WD        Abnorml    140000
4   5          60       RL  ...        WD         Normal    250000

[5 rows x 81 columns]

In [20]:
train.shape


(1460, 81)

In [21]:
test.shape

(1459, 80)

In [22]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total   Percent
PoolQC         1453  0.995205
MiscFeature    1406  0.963014
Alley          1369  0.937671
Fence          1179  0.807534
FireplaceQu     690  0.472603
LotFrontage     259  0.177397
GarageCond       81  0.055479
GarageType       81  0.055479
GarageYrBlt      81  0.055479
GarageFinish     81  0.055479
GarageQual       81  0.055479
BsmtExposure     38  0.026027
BsmtFinType2     38  0.026027
BsmtFinType1     37  0.025342
BsmtCond         37  0.025342
BsmtQual         37  0.025342
MasVnrArea        8  0.005479
MasVnrType        8  0.005479
Electrical        1  0.000685
Utilities         0  0.000000

In [0]:
train = train.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence','Id'], axis=1)

In [24]:
train.shape

(1460, 76)

In [25]:
#stats of the predictor variable (saleprice)
train.SalePrice.describe().round(2)

count      1460.0
mean     180921.2
std       79442.5
min       34900.0
25%      129975.0
50%      163000.0
75%      214000.0
max      755000.0
Name: SalePrice, dtype: float64

In [26]:
#spotting and removing outliers
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] =16.0
fig_size[1] = 4.0

x =train['SalePrice']
plt.hist(x, density=True, bins=400)
plt.ylabel('SalePrice');

In [27]:
##analysing the distribution of sale price
def reject_outliers(SalePrice):
    filtered= [e for e in (train['SalePrice']) if (e < 450000)]
    return filtered

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] =16.0
fig_size[1] = 4.0

filtered = reject_outliers('SalePrice')
plt.hist(filtered, 50)
fig_size[0]=16.0
fig_size[1]=8.0
plt.show()

df_no_outliers = pd.DataFrame(filtered)
df_no_outliers.shape

(1446, 1)

In [0]:

train = train[train['SalePrice']<450000]

In [29]:
train.head()

MSSubClass MSZoning  LotFrontage  ...  SaleType SaleCondition SalePrice
0          60       RL         65.0  ...        WD        Normal    208500
1          20       RL         80.0  ...        WD        Normal    181500
2          60       RL         68.0  ...        WD        Normal    223500
3          70       RL         60.0  ...        WD       Abnorml    140000
4          60       RL         84.0  ...        WD        Normal    250000

[5 rows x 76 columns]

In [0]:
X_train = train.drop(['SalePrice'], axis=1)

In [31]:

X_train.shape

(1446, 75)

In [32]:
import seaborn as sns
corr_matrix = X_train.corr()
f, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(corr_matrix, vmax=1, annot=True, square=True);
plt.show()

In [0]:
y_train = train['SalePrice']

In [34]:
y_train.shape

(1446,)

In [0]:
#y_train.describe()

In [36]:

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1446 entries, 0 to 1459
Data columns (total 75 columns):
MSSubClass       1446 non-null int64
MSZoning         1446 non-null object
LotFrontage      1188 non-null float64
LotArea          1446 non-null int64
Street           1446 non-null object
LotShape         1446 non-null object
LandContour      1446 non-null object
Utilities        1446 non-null object
LotConfig        1446 non-null object
LandSlope        1446 non-null object
Neighborhood     1446 non-null object
Condition1       1446 non-null object
Condition2       1446 non-null object
BldgType         1446 non-null object
HouseStyle       1446 non-null object
OverallQual      1446 non-null int64
OverallCond      1446 non-null int64
YearBuilt        1446 non-null int64
YearRemodAdd     1446 non-null int64
RoofStyle        1446 non-null object
RoofMatl         1446 non-null object
Exterior1st      1446 non-null object
Exterior2nd      1446 non-null object
MasVnrType       1439 no

In [0]:
#39 categorical values remain, the rest are numerical. These are the categrocial values:
cat_values= ['FireplaceQu','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
             'BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition']

In [0]:

#Missing data now replace by their mean or with categories by the most frequent value in each column.
X_train = X_train.apply(lambda x:x.fillna(x.value_counts().index[0])) #= fills every numercial column with its own most frequent value

X_train = X_train.fillna(X_train['GarageFinish'].value_counts().index[0]) #fill NaNs with the most frequent value from that column.
X_train = X_train.fillna(X_train['BsmtQual'].value_counts().index[0])
X_train = X_train.fillna(X_train['GarageType'].value_counts().index[0])
X_train = X_train.fillna(X_train['GarageQual'].value_counts().index[0])
X_train = X_train.fillna(X_train['GarageCond'].value_counts().index[0])
X_train = X_train.fillna(X_train['BsmtCond'].value_counts().index[0])
X_train = X_train.fillna(X_train['BsmtExposure'].value_counts().index[0])
X_train = X_train.fillna(X_train['BsmtFinType1'].value_counts().index[0])
X_train = X_train.fillna(X_train['FireplaceQu'].value_counts().index[0])

In [0]:
#OneHotEncoder to handle text and categorical attributes.
X_train = pd.get_dummies(X_train, columns=['FireplaceQu','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
                                           'ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition'])  

In [40]:
X_train.describe()

MSSubClass  LotFrontage  ...  SaleCondition_Normal  SaleCondition_Partial
count  1446.000000  1446.000000  ...           1446.000000            1446.000000
mean     56.995159    67.996542  ...              0.823651               0.082296
std      42.446131    22.138085  ...              0.381248               0.274910
min      20.000000    21.000000  ...              0.000000               0.000000
25%      20.000000    60.000000  ...              1.000000               0.000000
50%      50.000000    63.000000  ...              1.000000               0.000000
75%      70.000000    78.000000  ...              1.000000               0.000000
max     190.000000   313.000000  ...              1.000000               1.000000

[8 rows x 275 columns]

In [41]:
train.SalePrice.describe().round(2)

count      1446.00
mean     177268.66
std       69949.61
min       34900.00
25%      129900.00
50%      162000.00
75%      212000.00
max      446261.00
Name: SalePrice, dtype: float64

In [0]:
X_train = X_train.drop(['Condition2_RRAe',
                        'Exterior2nd_Other','Condition2_RRAn','Condition2_RRNn','HouseStyle_2.5Fin','RoofMatl_ClyTile','RoofMatl_Membran','RoofMatl_Metal','RoofMatl_Roll','Exterior1st_ImStucc','Heating_Floor','Heating_OthW','Electrical_Mix','GarageQual_Ex', 'Exterior1st_Stone','Utilities_NoSeWa'], axis=1)

In [43]:

X_train.shape

(1446, 259)

In [0]:
#df_test=pd.read_csv(test.csv)

In [45]:
test.shape

(1459, 80)

In [46]:

total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total   Percent
PoolQC         1456  0.997944
MiscFeature    1408  0.965045
Alley          1352  0.926662
Fence          1169  0.801234
FireplaceQu     730  0.500343
LotFrontage     227  0.155586
GarageCond       78  0.053461
GarageQual       78  0.053461
GarageYrBlt      78  0.053461
GarageFinish     78  0.053461
GarageType       76  0.052090
BsmtCond         45  0.030843
BsmtQual         44  0.030158
BsmtExposure     44  0.030158
BsmtFinType1     42  0.028787
BsmtFinType2     42  0.028787
MasVnrType       16  0.010966
MasVnrArea       15  0.010281
MSZoning          4  0.002742
BsmtHalfBath      2  0.001371

In [0]:

test = test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)

In [48]:
test.shape

(1459, 76)

In [0]:
test = test.apply(lambda x:x.fillna(x.value_counts().index[0])) #= fills every column with its own most frequent value

test = test.fillna(test['GarageFinish'].value_counts().index[0]) #fill NaNs with the most frequent value from that column.
test = test.fillna(test['BsmtQual'].value_counts().index[0])
test = test.fillna(test['FireplaceQu'].value_counts().index[0])
test = test.fillna(test['GarageType'].value_counts().index[0])
test = test.fillna(test['GarageQual'].value_counts().index[0])
test = test.fillna(test['GarageCond'].value_counts().index[0])
test = test.fillna(test['GarageFinish'].value_counts().index[0])
test = test.fillna(test['BsmtCond'].value_counts().index[0])
test = test.fillna(test['BsmtExposure'].value_counts().index[0])
test = test.fillna(test['BsmtFinType1'].value_counts().index[0])
test = test.fillna(test['BsmtFinType2'].value_counts().index[0])
test = test.fillna(test['BsmtUnfSF'].value_counts().index[0])

In [0]:
test = pd.get_dummies(test, columns=['FireplaceQu','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond',
                                        'Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition'])

In [51]:
test.shape

(1459, 260)

In [52]:
test.head()

Id  MSSubClass  ...  SaleCondition_Normal  SaleCondition_Partial
0  1461          20  ...                     1                      0
1  1462          20  ...                     1                      0
2  1463          60  ...                     1                      0
3  1464          60  ...                     1                      0
4  1465         120  ...                     1                      0

[5 rows x 260 columns]

In [0]:
test = test.drop(['Id'], axis=1)

In [54]:
test.shape

(1459, 259)

In [0]:
from xgboost import XGBRegressor

In [56]:
xgb_clf = XGBRegressor(n_estimators=1000, learning_rate=0.05)

xgb_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[10:21:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [57]:

xgb_predictions_test = xgb_clf.predict(test) # shape(1459, 262)

xgb_predictions_test

array([122130.336, 162974.27 , 186210.31 , ..., 151103.97 , 120057.55 ,
       227699.48 ], dtype=float32)

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
est = GradientBoostingRegressor(n_estimators= 1000, max_depth= 2, learning_rate= .01)
model=est.fit(X_train, y_train)

In [0]:
y_train_predict = est.predict(X_train)

In [0]:
y_test_predict = est.predict(test)

In [0]:
predictions = model.predict(test)